In [ ]:
import ctypes

import tkinter as tk
from tkinter import ttk

from ctypes import wintypes, POINTER, Structure

# Define the GUID structure as per Windows API
class GUID(Structure):
    _fields_ = [
        ("Data1", wintypes.DWORD),
        ("Data2", wintypes.WORD),
        ("Data3", wintypes.WORD),
        ("Data4", wintypes.BYTE * 8)
    ]

# Windows API functions and constants
powrprof = ctypes.WinDLL('powrprof.dll')

# Function definitions
POWER_WRITE_AC_VALUE_INDEX = powrprof.PowerWriteACValueIndex
POWER_WRITE_AC_VALUE_INDEX.argtypes = [wintypes.HANDLE, POINTER(GUID), POINTER(GUID), POINTER(GUID), wintypes.DWORD]
POWER_WRITE_AC_VALUE_INDEX.restype = wintypes.DWORD

# Function to set processor power management
def set_processor_power(percent):
    # Placeholder values - replace these with the actual min/max values for the setting
    maximum_value = 100
    minimum_value = 0

    # Check if the percent is within the allowed range
    if not (0 <= percent <= 100):
        raise ValueError("Percent value should be between 0 and 100")

    # Adjust the percent value based on the min/max range
    new_value = int((percent / 100) * (maximum_value - minimum_value) + minimum_value)
    
    # The GUIDs need to be correctly identified and set here
    power_plan_guid = GUID() # Replace with actual GUID
    processor_power_guid = GUID() # Replace with actual GUID
    #new_value = int(percent * (maximum_value - minimum_value) / 100) # Convert percent to the appropriate scale

    # Call the Windows API to set the new value
    result = POWER_WRITE_AC_VALUE_INDEX(None, ctypes.byref(power_plan_guid), None, ctypes.byref(processor_power_guid), new_value)

    # Check the result of the API call
    if result != 0:
        # If result is non-zero, it indicates an error
        error_code = ctypes.GetLastError()
        error_message = ctypes.FormatError(error_code)
        raise Exception(f"Failed to set power value. Error code: {error_code}, Message: {error_message}")
    else:
        print("Power value set successfully.")
# GUI
app = tk.Tk()
app.title("Processor Power Management")

# Slider for power percentage
power_slider = ttk.Scale(app, from_=0, to=100, orient="horizontal", command=lambda v: set_processor_power(float(v)))
power_slider.pack()

# Run the application
app.mainloop()

In [6]:
import subprocess

def set_processor_power(plan_name, setting_name, value):
    # Get the GUID of the power plan
    result = subprocess.run(["powercfg", "-l"], capture_output=True, text=True)
    lines = result.stdout.split('\n')
    plan_guid = None
    for line in lines:
        if plan_name in line:
            plan_guid = line.split()[3]
            break

    if not plan_guid:
        raise Exception("Power plan not found")

    # Set the processor power management to the desired value
    subprocess.run(["powercfg", "-setacvalueindex", plan_guid, "SUB_PROCESSOR", setting_name, str(value)])
    subprocess.run(["powercfg", "-setactive", plan_guid])

    print(f"Set {setting_name} to {value}% for '{plan_name}' plan")

# Example Usage
set_processor_power("Balanced", "PROCTHROTTLEMAX", 85)

Set PROCTHROTTLEMAX to 85% for 'Balanced' plan


In [8]:
import tkinter as tk
from tkinter import ttk
import subprocess

# Define a list of percentages
percentages = [30, 60, 100]
current_index = [0]  # Use a list for mutability in nested function

def set_processor_power(plan_name, setting_name, value):
    # Get the GUID of the power plan
    result = subprocess.run(["powercfg", "-l"], capture_output=True, text=True)
    lines = result.stdout.split('\n')
    plan_guid = None
    for line in lines:
        if plan_name in line:
            plan_guid = line.split()[3]
            break

    if not plan_guid:
        raise Exception("Power plan not found")

    # Set the processor power management to the desired value
    subprocess.run(["powercfg", "-setacvalueindex", plan_guid, "SUB_PROCESSOR", setting_name, str(value)])
    subprocess.run(["powercfg", "-setactive", plan_guid])

    print(f"Set {setting_name} to {value}% for '{plan_name}' plan")

def update_power_setting():
    global current_index
    current_index[0] = (current_index[0] + 1) % len(percentages)
    new_percentage = percentages[current_index[0]]
    set_processor_power("Balanced", "PROCTHROTTLEMAX", new_percentage)
    label.config(text=f"Current Setting: {new_percentage}%")

# Tkinter GUI setup
app = tk.Tk()
app.title("Processor Power Management")

# Add a label to show the current setting
label = tk.Label(app, text="Current Setting: 20%")
label.pack()

# Add a button to change the power setting
change_button = ttk.Button(app, text="Change Power Setting", command=update_power_setting)
change_button.pack()

# Run the application
app.mainloop()

Set PROCTHROTTLEMAX to 40% for 'Balanced' plan
Set PROCTHROTTLEMAX to 60% for 'Balanced' plan
Set PROCTHROTTLEMAX to 80% for 'Balanced' plan
Set PROCTHROTTLEMAX to 100% for 'Balanced' plan
Set PROCTHROTTLEMAX to 20% for 'Balanced' plan


In [2]:
import tkinter as tk
from tkinter import ttk
import subprocess
import re

# Define a list of percentages
percentages = [30, 60, 100]
current_index = [0]  # Use a list for mutability in nested function


# Function to get the GUID of the currently active power plan
def get_active_plan_guid():
    result = subprocess.run(["powercfg", "-getactivescheme"], capture_output=True, text=True)
    match = re.search(r'Power Scheme GUID: ([a-f0-9-]+)  \(.*\)', result.stdout)
    if match:
        return match.group(1)
    else:
        raise Exception("Failed to get the active power plan GUID")

# Function to get the current processor power management value
def get_current_processor_power_value(plan_guid, setting_name):
    # Query the current setting value
    result = subprocess.run(["powercfg", "-q", plan_guid, "SUB_PROCESSOR", setting_name], capture_output=True, text=True)
    match = re.search(r'Power Setting Index: (\d+)', result.stdout)
    if match:
        # Convert the value to percentage
        print(match)
        return int(match.group(1)) * 100 // 0x64
    else:
        raise Exception("Failed to get current power setting value")

def set_processor_power(plan_name, setting_name, value):
    # Get the GUID of the power plan
    result = subprocess.run(["powercfg", "-l"], capture_output=True, text=True)
    lines = result.stdout.split('\n')
    plan_guid = None
    for line in lines:
        if plan_name in line:
            plan_guid = line.split()[3]
            break

    if not plan_guid:
        raise Exception("Power plan not found")

    # Set the processor power management to the desired value
    subprocess.run(["powercfg", "-setacvalueindex", plan_guid, "SUB_PROCESSOR", setting_name, str(value)])
    subprocess.run(["powercfg", "-setdcvalueindex", plan_guid, "SUB_PROCESSOR", setting_name, str(value)])
    subprocess.run(["powercfg", "-setactive", plan_guid])

    print(f"Set {setting_name} to {value}% for '{plan_name}' plan")

def update_power_setting():
    global current_index
    current_index[0] = (current_index[0] + 1) % len(percentages)
    new_percentage = percentages[current_index[0]]
    set_processor_power("Balanced", "PROCTHROTTLEMAX", new_percentage)
    label.config(text=f"Current Setting: {new_percentage}%")

def start_move(event):
    app.x = event.x
    app.y = event.y

def stop_move(event):
    app.x = None
    app.y = None

def on_motion(event):
    deltax = event.x - app.x
    deltay = event.y - app.y
    x = app.winfo_x() + deltax
    y = app.winfo_y() + deltay
    app.geometry(f"+{x}+{y}")

def close_app():
    app.destroy()
    
# Tkinter GUI setup
app = tk.Tk()
app.title("Processor Power Management")

# Remove the title bar and make the window borderless
app.overrideredirect(True)

# Bind mouse events for dragging the window
app.bind('<Button-1>', start_move)
app.bind('<ButtonRelease-1>', stop_move)
app.bind('<B1-Motion>', on_motion)

# Add a button to change the power setting
change_button = ttk.Button(app, text="Change Power Setting", command=update_power_setting)
change_button.pack()

# Get the current active plan GUID and current setting
active_plan_guid = get_active_plan_guid()
current_value = get_current_processor_power_value(active_plan_guid, "PROCTHROTTLEMAX")

# Add a label to show the current setting
label = tk.Label(app, text=f"Current Setting: {current_value}")
label.pack()

# Add a close button
close_button = ttk.Button(app, text="Close", command=close_app)
close_button.pack()
# Run the application
app.mainloop()

<re.Match object; span=(514, 536), match='Power Setting Index: 0'>


In [16]:
active_plan_guid

'381b4222-f694-41f0-9685-ff5bb260df2e'

In [17]:
current_value

0

In [11]:
#powercfg -q 381b4222-f694-41f0-9685-ff5bb260df2e "SUB_PROCESSOR" "PROCTHROTTLEMAX"

# Query the current setting value
result = subprocess.run(["powercfg", "-q", "381b4222-f694-41f0-9685-ff5bb260df2e", "SUB_PROCESSOR", "PROCTHROTTLEMAX"], capture_output=True, text=True)
match = re.search(r'Power Setting Index: 0x([\da-fA-F]+)', result.stdout)
if match:
    # Convert the value to percentage
    print(match)
    #int(match.group(1)) * 100 // 0x64

<re.Match object; span=(514, 545), match='Power Setting Index: 0x00000064'>
